10538번 빅 픽쳐 <span style="color:red">실패</span> - 2025.09.18

In [ ]:
# import sys
# input = sys.stdin.readline

# -------------------------------
# 상수 정의 (모듈러, 베이스)
# -------------------------------
# 충돌을 더 줄이기 위해 두 개의 서로 다른 큰 소수(mod)를 사용 (이중 해시)
MOD1 = 10**9 + 7
MOD2 = 10**9 + 9

# 2D 해시에서 행 방향과 열 방향에 사용할 "베이스" 값들
# (보통은 작은 값도 쓰지만 이미 정해진 상수 사용 가능)
BASE_ROW = 911382323   # 세로(행) 방향의 베이스
BASE_COL = 972663749   # 가로(열) 방향의 베이스

# -------------------------------
# 2D prefix 해시 테이블 생성 함수
# -------------------------------
def build_hash(matrix, h, w):
    """
    matrix: 문자 그리드 (리스트 of 문자열)
    h: matrix의 행 수
    w: matrix의 열 수

    반환값: (h+1) x (w+1) 크기의 2D prefix 해시 테이블
    h[i][j] 는 (0,0)부터 (i-1, j-1) 까지의 블록 전체 해시 (튜플: (mod1, mod2))
    -> 내부적으로 1-based 루프를 사용하므로 hash_list[0][*], hash_list[*][0]은 0(패딩)
    """
    # (h+1) x (w+1) 크기: 0번째 행/열은 패딩(0)으로 둠
    hash_list = [[(0, 0) for _ in range(w + 1)] for _ in range(h + 1)]

    # i, j 루프는 1부터 시작해서 matrix[i-1][j-1]을 읽음 (0-based 원본 문자열)
    for i in range(1, h + 1):
        row_hash = (0, 0)  # 현재 행에서 왼쪽에서 오른쪽으로 누적되는 해시
        for j in range(1, w + 1):
            # 문자 -> 정수 매핑: 'x' = 1, 'o' = 2 (0이면 구분이 안 됨으로 0이 아닌 값)
            val = 1 if matrix[i - 1][j - 1] == 'x' else 2

            # 1) 같은 행 안에서의 가로방향 rolling hash 갱신
            # row_hash = row_hash * BASE_COL + val  (각 칸의 가로 위치 반영)
            row_hash = ((row_hash[0] * BASE_COL + val) % MOD1,
                        (row_hash[1] * BASE_COL + val) % MOD2)

            # 2) 세로 누적: 위쪽 블록 해시에 현재 행의 row_hash를 결합
            # h[i][j] = h[i-1][j] * BASE_ROW + row_hash
            # (즉, 행 방향(위->아래)에도 베이스를 곱해 위치를 반영)
            up = hash_list[i - 1][j]
            hash_list[i][j] = ((up[0] * BASE_ROW + row_hash[0]) % MOD1,
                               (up[1] * BASE_ROW + row_hash[1]) % MOD2)
    return hash_list

# -------------------------------
# 부분 블록(사각형) 해시 추출 함수
# -------------------------------
def get_subhash(hash_list, x1, y1, x2, y2, pow_r, pow_c):
    """
    hash_list: build_hash에서 생성한 prefix 해시 테이블
    (x1, y1): 부분 블록의 왼쪽위 좌표 (0-based 기준, inclusive)
    (x2, y2): 부분 블록의 오른쪽아래 +1 좌표 (exclusive) -> (x2-1, y2-1) 까지 포함
      즉 블록의 크기는 높이 = x2-x1, 너비 = y2-y1
    pow_r: 행(세로) 베이스의 거듭제곱 튜플 리스트 (index = exponent) : [(1,1), (BASE_ROW % MOD1, BASE_ROW % MOD2), ...]
    pow_c: 열(가로) 베이스의 거듭제곱 튜플 리스트

    반환값: (mod1값, mod2값) 형태의 부분 블록 해시
    """
    # prefix 해시에서 네 꼭짓점 값을 꺼낸다
    a1, a2 = hash_list[x2][y2]  # (0,0) -> (x2-1,y2-1) 전체 해시
    b1, b2 = hash_list[x1][y2]  # (0,0) -> (x1-1,y2-1) (위쪽 제거 영역)
    c1, c2 = hash_list[x2][y1]  # (0,0) -> (x2-1,y1-1) (왼쪽 제거 영역)
    d1, d2 = hash_list[x1][y1]  # (0,0) -> (x1-1,y1-1) (위쪽+왼쪽 중복 영역)

    # 블록의 너비(가로 길이), 높이(세로 길이)
    w, h = y2 - y1, x2 - x1

    # inclusion-exclusion 공식 (2D prefix 해시에서 부분 행렬의 해시를 뽑는 식)
    # val = A - B * BASE_ROW^h - C * BASE_COL^w + D * BASE_ROW^h * BASE_COL^w  (mod)
    # 여기서 pow_r[h][0]는 BASE_ROW^h (mod MOD1), pow_c[w][0]는 BASE_COL^w (mod MOD1)
    val1 = (a1 - b1 * pow_r[h][0] % MOD1
               - c1 * pow_c[w][0] % MOD1
               + d1 * pow_r[h][0] % MOD1 * pow_c[w][0] % MOD1) % MOD1

    val2 = (a2 - b2 * pow_r[h][1] % MOD2
               - c2 * pow_c[w][1] % MOD2
               + d2 * pow_r[h][1] % MOD2 * pow_c[w][1] % MOD2) % MOD2

    return (val1, val2)

# -------------------------------
# 메인 (입력 처리 및 탐색)
# -------------------------------
h_p, w_p, h_m, w_m = map(int, input().split())
pattern = [input().strip() for _ in range(h_p)]  # 찾을 작은 그림 (hp x wp)
master = [input().strip() for _ in range(h_m)]   # 큰 그림 (hm x wm)

# -------------------------------
# 거듭제곱 테이블 미리 계산
# pow_r[k] = (BASE_ROW^k mod MOD1, BASE_ROW^k mod MOD2)
# pow_c[k] = (BASE_COL^k mod MOD1, BASE_COL^k mod MOD2)
# -------------------------------
pow_r = [(1, 1)]
for i in range(h_m):  # 최대 높이 hm 까지 필요 (부분 블록의 높이는 최대 hm)
    pow_r.append(((pow_r[-1][0] * BASE_ROW) % MOD1,
                  (pow_r[-1][1] * BASE_ROW) % MOD2))

pow_c = [(1, 1)]
for i in range(w_m):  # 최대 너비 wm 까지 필요
    pow_c.append(((pow_c[-1][0] * BASE_COL) % MOD1,
                  (pow_c[-1][1] * BASE_COL) % MOD2))

# -------------------------------
# 2D prefix 해시 생성 (패턴과 마스터)
# - 두 테이블 모두 (rows+1) x (cols+1) 크기 (0번째 행/열은 0)
# -------------------------------
h_pattern = build_hash(pattern, h_p, w_p)
h_master  = build_hash(master,  h_m, w_m)

# 패턴 전체(0,0)~(hp-1,wp-1) 블록의 해시 == target
# get_subhash는 (x1,y1,x2,y2)에서 x2,y2는 exclusive 이므로 (0,0,hp,wp)로 호출
target = get_subhash(h_pattern, 0, 0, h_p, w_p, pow_r, pow_c)

# -------------------------------
# 마스터에서 가능한 모든 시작 위치에 대해 패턴 해시와 비교
# 시작 좌표 i는 0..hm-hp, j는 0..wm-wp (0-based)
# -------------------------------
ans = 0
for i in range(h_m - h_p + 1):
    for j in range(w_m - w_p + 1):
        # 마스터의 (i,j)에서 (i+hp-1, j+wp-1) 블록 해시
        block = get_subhash(h_master, i, j, i + h_p, j + w_p, pow_r, pow_c)
        if block == target:
            # 해시 일치 -> (거의 항상) 패턴 일치
            # (충돌 가능성 희박하지만 엄밀히 비교하려면 문자 비교 추가)
            ans += 1

print(ans)

# -------------------------------
# 추가 설명(요약)
# - hash_list는 내부적으로 1-based 인덱스 논리를 사용하지만,
#   matrix 접근은 0-based(matrix[i-1][j-1])로 이루어진다.
# - get_subhash의 (x1,y1)과 (x2,y2)는 모두 0-based 좌표다.
#   반환 블록은 (x1..x2-1, y1..y2-1).
# - pow_r, pow_c는 거듭제곱(행/열 베이스)의 튜플 리스트로,
#   인덱스는 거듭제곱 차수(=블록 높이 또는 너비)이다.
# - 시간 복잡도: 해시 생성 O(hm * wm), 블록 비교 O((hm-hp+1)*(wm-wp+1)).
# - 충돌 걱정 완화: 두 모듈러(MOD1, MOD2) 사용.
# -------------------------------
